In [1]:
import os
import random
import pandas as pd
from tqdm import tqdm

In [2]:
from NetworkGen import *

In [3]:
parent_dir = os.path.dirname(os.getcwd())
os.makedirs(parent_dir + '/Results', exist_ok=True)

# Generate networks

## Train

In [4]:
# Initialize directory to save network structures
data = "Train"
network_dir = os.path.join(parent_dir, "Networks", data)
os.makedirs(network_dir, exist_ok=True)
# Desired network structures
train_networks = {1: [5, 10, 25, 1],
                  3: [8, 27, 50, 1]}
replications = {1: 10, 3: 10}

In [ ]:
# Generate train networks
for id, nNodes in tqdm(train_networks.items()):
    for k in range(1, replications[id]+1):
        G = hierarchical_network(nNodes, seed=10+k)
        # Add edge attributes
        random.seed(10+k)
        c_u = 20
        for u,v in G.edges:
            G[u][v]["c"] = random.randint(1, 20)
            G[u][v]["b"] = 1
        save_network_structure(G, f"{network_dir}/network_{id}_{k}")

In [5]:
# Save the networks details
cols = ["id", "k", "|N|", "|A|", "Level 0", "Level 1", "Level 2", "Level 3", "Level 4"]
vals = []
for f in tqdm(os.listdir(network_dir)):
    name, ext = f.split('.')
    G = load_network(f"{network_dir}/{name}")
    id, k = map(int, name.split('_')[1:])
    n, m = G.number_of_nodes(), G.number_of_edges()
    row = [id, k, n, m, 1]
    row += train_networks[id]
    vals.append(row)
train_networks_df = pd.DataFrame(vals, columns=cols)
train_networks_df = train_networks_df.sort_values(by=["id", "k"])
train_networks_df.to_csv(f"Results/TrainNetworksDetails.csv", index=False)

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 98.01it/s]


## Test

In [5]:
# Initialize directory to save network structures
data = "Test"
network_dir = os.path.join(parent_dir, "Networks", data)
os.makedirs(network_dir, exist_ok=True)

In [6]:
# Desired network structures and number of replications
small_networks_h = {1: [5, 10, 25, 1],
                    3: [8, 27, 50, 1],
                    5: [30, 100, 450, 1]}
large_networks_h = {6: [1000, 1500, 1500, 2000, 2500, 2500, 3000, 5000, 10000, 1],
                    7: [2500, 3000, 3000, 4500, 4500, 7000, 7000, 10000, 15000, 1],
                    8: [2500, 3000, 3000, 4500, 4500, 7000, 7000, 10000, 15000, 1]}
networks_nh = {2: [5, 10, 25, 40, 1],
               4: [12, 50, 200, 410, 1]}
replications = {1: 10, 3: 10, 5: 10, 6: 1, 7: 1, 8: 1, 2: 10, 4: 10}

In [7]:
# Arc flow capacity and interdiction cost
large_networks_c = {0: (50,100), 1: (1,75), 2: (1,50), 3: (1,40), 4: (1,40), 5: (1,30), 6: (1,30), 7: (1,20), 8: (1,20), 9: (50,100)}
large_networks_b = {0: (100,101), 1: (1,75), 2: (1,50), 3: (1,40), 4: (1,40), 5: (1,30), 6: (1,30), 7: (1,20), 8: (1,20), 9: (100,101)}

In [8]:
# Generate small heierarchical networks
for id, nNodes in tqdm(small_networks_h.items()):
    for k in range(1, replications[id]+1):
        G = hierarchical_network(nNodes, seed=k)
        # Add edge attributes
        random.seed(k)
        if k <= 5:
            c_u = 20
        else:
            c_u = 50
        for u,v in G.edges:
            G[u][v]["c"] = random.randint(1, c_u)
            G[u][v]["b"] = random.randint(1, 50)
        save_network_structure(G, f"{network_dir}/network_{id}_{k}")

In [9]:
# Generate large heierarchical networks
for id, nNodes in tqdm(large_networks_h.items()):
    for k in range(1, replications[id]+1):
        G = hierarchical_network(nNodes, seed=k)# Add edge attributes
        random.seed(k)
        for u,v in G.edges:
            u_level = G.nodes[u]['subset']
            G[u][v]["c"] = random.randint(large_networks_c[u_level][0], large_networks_c[u_level][1])
            G[u][v]["b"] = random.randint(large_networks_b[u_level][0], large_networks_b[u_level][1])
        save_network_structure(G, f"{network_dir}/network_{id}_{k}")

In [10]:
# Generate large heierarchical networks
for id, nNodes in tqdm(networks_nh.items()):
    for k in range(1, replications[id]+1):
        G = non_hierarchical_network(nNodes, seed=k)
        # Add edge attributes
        random.seed(k)
        if k <= 5:
            c_u = 20
        else:
            c_u = 50
        for u,v in G.edges:
            G[u][v]["c"] = random.randint(1, c_u)
            G[u][v]["b"] = random.randint(1, 50)
        save_network_structure(G, f"{network_dir}/network_{id}_{k}")

In [8]:
# Split newtorks into small and large
small_test_networks = {**small_networks_h, **networks_nh}
large_test_networks = {**large_networks_h}

In [10]:
# Save the small networks details
cols = ["id", "k", "|N|", "|A|", "Level 0", "Level 1", "Level 2", "Level 3", "Level 4", "Level 5"]
vals = []
for f in tqdm(os.listdir(network_dir)):
    name, ext = f.split('.')
    id, k = map(int, name.split('_')[1:])
    if id in small_test_networks:
        G = load_network(f"{network_dir}/{name}")
        n, m = G.number_of_nodes(), G.number_of_edges()
        row = [id, k, n, m, 1]
        row += small_test_networks[id]
        if id % 2 != 0:
            row += [0]
        vals.append(row)
small_test_networks_df = pd.DataFrame(vals, columns=cols).replace({0: '-'})
small_test_networks_df = small_test_networks_df.sort_values(by=["id", "k"])
small_test_networks_df.to_csv(f"Results/SmallTestNetworksDetails.csv", index=False)

100%|██████████| 53/53 [00:02<00:00, 19.62it/s] 


In [12]:
# Save the large networks details
cols = ["id", "k", "|N|", "|A|", "Level 0", "Level 1", "Level 2", "Level 3", "Level 4", "Level 5", "Level 6", "Level 7", "Level 8", "Level 9", "Level 10"]
vals = []
for f in tqdm(os.listdir(network_dir)):
    name, ext = f.split('.')
    id, k = map(int, name.split('_')[1:])
    if id in large_test_networks:
        G = load_network(f"{network_dir}/{name}")
        n, m = G.number_of_nodes(), G.number_of_edges()
        row = [id, k, n, m]
        # At each level we have [nNodes/c_range/b_range]
        for i in range(10):
            row.append(f"[{large_test_networks[id][i]}/{large_networks_c[i]}/{large_networks_b[i]}]")
        row.append(f"[{1}/-/-]")
        vals.append(row)
large_test_networks_df = pd.DataFrame(vals, columns=cols)
large_test_networks_df = large_test_networks_df.sort_values(by=["id", "k"])
large_test_networks_df.to_csv(f"Results/LargeTestNetworksDetails.csv", index=False)

100%|██████████| 53/53 [00:24<00:00,  2.15it/s]
